In [1]:
# from langchain_openai import AzureChatOpenAI
# from langchain.prompts import ChatPromptTemplate
# import os

# import truststore
# truststore.inject_into_ssl()
# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# import time
# import seaborn as sns
# import numpy as np
# from sklearn.metrics import roc_curve, auc
# import matplotlib.pyplot as plt
# os.environ["APP_CLIENT_ID"] = "zizhang-chen-research-app"
# os.environ["APP_CLIENT_SECRET"] = "cQPcueFP7tDrimbf8NW2GAHcHeQa"
# from llm_idam_token_generator.idam_token_generator import get_idam_token
# from langchain_community.callbacks import get_openai_callback
# from langchain_openai import OpenAI
# import tiktoken

from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
import os

# import truststore
# truststore.inject_into_ssl()
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import seaborn as sns
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
# os.environ["APP_CLIENT_ID"] = "zizhang-chen-research-app"
# os.environ["APP_CLIENT_SECRET"] = "cQPcueFP7tDrimbf8NW2GAHcHeQa"
# from llm_idam_token_generator.idam_token_generator import get_idam_token
from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI
import tiktoken


In [2]:
# end_point_list = [
#     'https://lmaas-beta.ai.gehealthcare.com',
#     "https://openai-llm-frontdoor-hma7evbthrd4cugn.a01.azurefd.net"
# ]

# model_list = ["gpt-35-turbo-16k", "gpt-4-32k-beta", "gpt-4o"]

# # APIM_KEY = "8b96051ed6b84e4dad762fdc9f8c809e"

# OPENAI_ENDPOINT = end_point_list[0]
# OPENAI_DEPLOYMENT_MODEL = model_list[1]
# OPENAI_AZURE_API_VERSION = "2023-12-01-preview"
# # OPENAI_AZURE_API_VERSION = '2024-02-01'
# OPENAI_TYPE="azure"

In [3]:
# llm = AzureChatOpenAI(
#     api_key=
#     "xxx",  # This is not playing any role, but required as per OpenAI sdk. So any random could be passed.
#     azure_endpoint=OPENAI_ENDPOINT,
#     deployment_name=OPENAI_DEPLOYMENT_MODEL,
#     openai_api_version=OPENAI_AZURE_API_VERSION,
#     n=2,
#     default_headers={
#         'Authorization': f'Bearer {get_idam_token()}',
#         'Content-Type': 'application/json'
#     })
# # prompt = ChatPromptTemplate.from_template("what is the city {person} is from?")
# # chain = prompt | llm
# # print(chain.invoke({"person": "Narendra modi"}))

In [4]:
df = pd.read_csv('data/new_diagnose_patient_description.csv')
description_list = df['description'].values
new_diagnose_list = ["value_new_hypertension", "value_new_hyperlipidemia", "value_new_acutemi"]
# new_diagnose_list = [i.split('value_')[1] for i in df.columns[2:8]]
# new_diagnose_prediction = [
#     "Predict whether the patient will have first diagnosis of: 'hypertension', within the next year after discharge from hospital.",
#     "Predict whether the patient will have first diagnosis of: 'hyperlipidemia', within the next year after discharge from hospital.",
#     "Predict whether the patient will have first diagnosis of: 'pancreatic cancer', within the next year after discharge from hospital.",
#     "Predict whether the patient will have first diagnosis of: 'celiac disease' within the next year after discharge from hospital.",
#     "Predict whether the patient will have first diagnosis of disease: 'lupus', within the next year after discharge from hospital.",
#     "Predict whether the patient will have first diagnosis of disease: 'acute myocardial infarction', within the next year after discharge from hospital."
# ]

new_diagnose_prediction = [
        "Predict whether the patient will have first diagnosis of: 'Hypertension', within the next year after discharge from hospital.",
        "Predict whether the patient will have first diagnosis of: 'Hyperlipidemia', within the next year after discharge from hospital.",
        "Predict whether the patient will have first diagnosis of disease: 'Acute Myocardial Infarction', within the next year after discharge from hospital."
    ]

In [10]:
def split_string_into_two_parts(text):
    lines = text.split('\n')
    first_part = lines[:2]
    second_part = lines[2:]

    first_part_string = '\n'.join(first_part)
    second_part_string = '\n'.join(second_part)

    return first_part_string, second_part_string

def combin_prompt_cot(general_prompt, person_info, medical_event, template_events_answer_style):
    template_by_task = '**Task:**\n' + general_prompt + '\n'

    template_info = '**Patient age and demographic information:**\n' + person_info + '\n'

    template_events = '**Medical Events:**\n' + medical_event + '\n'

    return template_by_task + '\n' + '{events}' , template_info + '\n' + template_events + template_events_answer_style

def calculate_row_entropy(df):
    def entropy(row):
        counts = row.value_counts(normalize=True)
        return -np.sum(counts * np.log2(counts + np.finfo(float).eps))

    return df.apply(entropy, axis=1)

def num_tokens_from_string(string, encoding_model = 'gpt-3.5-turbo') -> int:
    encoding = tiktoken.encoding_for_model(encoding_model)
    num_tokens = len(encoding.encode(string))
    return num_tokens


def generate_anwer_requirement():
    template_events_answer_style = "**Answer requirement:**\n"

    answer_requirements = "Please only answer with 'Yes' or 'No'"

    # new_diagnose_prediction = [
    #     "Predict whether the patient will have first diagnosis of disease: 'hypertension', within the next year after discharge from hospital.",
    #     "Predict whether the patient will have first diagnosis of disease: 'hyperlipidemia', within the next year after discharge from hospital.",
    #     "Predict whether the patient will have first diagnosis of disease: 'pancreatic cancer', within the next year after discharge from hospital.",
    #     "Predict whether the patient will have first diagnosis of disease: 'celiac', within the next year after discharge from hospital.",
    #     "Predict whether the patient will have first diagnosis of disease: 'lupus', within the next year after discharge from hospital.",
    #     "Predict whether the patient will have first diagnosis of disease: 'acute myocardial infarction', within the next year after discharge from hospital."
    # ]

    new_diagnose_prediction = [
        "Predict whether the patient will have first diagnosis of disease: 'Hypertension', within the next year after discharge from hospital.",
        "Predict whether the patient will have first diagnosis of disease: 'Hyperlipidemia', within the next year after discharge from hospital.",
        "Predict whether the patient will have first diagnosis of disease: 'Acute Myocardial Infarction', within the next year after discharge from hospital."
    ]

    modified_list = [
        s + " " + answer_requirements for s in new_diagnose_prediction
    ]

    final_string_requirements = '\n'.join(modified_list)

    final_string_requirements = template_events_answer_style + final_string_requirements

    example_answer = "\n\n**Answer example:**\n"

    # example_prediction = "hypertension: Yes\n" + "hyperlipidemia: Yes\n" + "pancreatic cancer: No\n" + "celiac: No\n" + "lupus: No\n" + "acute myocardial infarction: No"

    example_prediction = "Hypertension: Yes\n" + "Hyperlipidemia: Yes\n" + "Acute Myocardial Infarction: No"

#     final_string_example = example_answer + example_prediction

#     final_string = final_string_requirements + final_string_example
    final_string = final_string_requirements
    return final_string

def generate_prompt_cot():
    cot_prompt = ("")
    # cot_prompt = (
    #     "**Chain of Thought Analysis:**\n"
    #     "Given the patient's full medical profile, including their recent lab results and documented medical history, "
    #     "analyze the likelihood of several key diagnoses. Critically evaluate how the confidence in diagnosing each condition "
    #     "might influence the assessment of other interrelated conditions. Here's how to proceed:\n"
    #     "1. **Evaluate Hypertension**: Reflect on the confidence of diagnosing hypertension based on the patient's blood pressure "
    #     "trends and related factors. Consider how this diagnosis, if made with high confidence, might influence the confidence "
    #     "in diagnosing cardiovascular-related conditions such as hyperlipidemia and acute myocardial infarction.\n"
    #     "2. **Assess Hyperlipidemia**: Determine the certainty of the hyperlipidemia diagnosis by considering lipid profiles "
    #     "and genetic factors. Discuss how strong evidence for hyperlipidemia could enhance the diagnostic confidence for "
    #     "acute myocardial infarction due to shared risk pathways.\n"
    #     "3. **Consider Pancreatic Cancer (Pancan)**: Judge the likelihood of pancreatic cancer, taking into account symptoms "
    #     "and biomarkers. Explain how the confidence in diagnosing related conditions like diabetes could affect the certainty "
    #     "of diagnosing pancreatic cancer.\n"
    #     "4. **Diagnose Celiac Disease**: Evaluate the strength of the celiac disease diagnosis through clinical presentation "
    #     "and antibody testing. Reflect on how a firm diagnosis here might impact the diagnostic confidence in other autoimmune "
    #     "diseases such as lupus.\n"
    #     "5. **Examine Lupus**: Look into lupus based on symptomatology and immunological tests. Analyze how diagnosing other "
    #     "autoimmune conditions with high confidence, like celiac disease, might affect the certainty in diagnosing lupus.\n"
    #     "6. **Predict Acute Myocardial Infarction (Acute MI)**: Consider the robustness of evidence for acute myocardial "
    #     "infarction by assessing cardiac risk factors and symptoms. Discuss how high confidence in hypertension and hyperlipidemia "
    #     "diagnoses might reinforce the diagnosis of acute MI.\n"
    #     "Conclude by integrating these assessments to discuss how confidence in one diagnosis might substantively influence the "
    #     "confidence in others, leading to a more nuanced understanding of the patient’s overall health."
    # )
    return cot_prompt


def generate_prompt_task(new_diagnose_prediction):
    multi_task_prompt = '\n'.join(new_diagnose_prediction)
    content_1 = f"You are an experienced doctor. Based on the provided patient age, demographic information and medical events, use your medical knowledge and reasoning to:\n{multi_task_prompt}\n"
    content_2 = f"You are give the medical events up to the patient's discharges. Please respond 'Yes' if it is plausible and 'No' otherwise. Be aware that your confidence in diagnosis one disease might substantively influence the confidence in other diseases.\n"
    # content_3 = f"Notice that:\nHyperlipidemia and Hypertension are well-known risk factors for cardiovascular diseases, including Acute Myocardial Infarction. Also, Hyperlipidemia and Hypertension are two cardiovascular risk factors that often coexist and are interrelated through various mechanisms. "
    content_3 = f"Notice that:\nHyperlipidemia and Hypertension are well-known risk factors for cardiovascular diseases, including Acute Myocardial Infarction."
    return content_1 + content_2 + content_3

In [11]:
out_csv_names = 'multi_task_lab_test.csv'
general_prompt = generate_prompt_task(new_diagnose_prediction)
# gt_list = df.loc[:, ['value_' + new_diagnose_list[i] for i in range(len(new_diagnose_list))]].values
gt_list = df.loc[:, new_diagnose_list]
gt_list = pd.DataFrame(gt_list)
with get_openai_callback() as cb:

    answer_list_all = []

    for i in tqdm(range(len(df))):
        # for i in tqdm(range(50, 100)):
        # for i in tqdm(range(1)):
        row = df.loc[i, :]
        answer_list_5 = []
        for _ in range(5):
            # for _ in range(1):
            current_description = description_list[i]
            person_info, medical_events = split_string_into_two_parts(
                current_description)
            anwer_requirement = generate_anwer_requirement()
            c, e = combin_prompt_cot(general_prompt, person_info,
                                     medical_events, anwer_requirement)
    #         prompt = ChatPromptTemplate.from_template(c)
    #         chain = prompt | llm
    #         result = chain.invoke({'events': e})
    #         answer_list_5.append(result.content)
    #     answer_list_all.append(answer_list_5)
    # df_pred = pd.DataFrame(answer_list_all)
    # df_pred.columns = ['pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5']
    # df_pred = pd.concat([df_pred, gt_list], axis=1)

    # # df_pred.to_csv(os.path.join('results/gpt35_new/cross_task_new_diagnose/',
    # #                             out_csv_names),
    # #                index=False)
    # # df_pred.to_csv(os.path.join('results/gpt4_new/cross_task_new_diagnose/', out_csv_names), index=False)

    # # df_pred.to_csv(os.path.join('results/gpt35_new/cross_task_new_diagnose_blood/', out_csv_names), index=False)

    # df_pred.to_csv(os.path.join('results/gpt4_new/cross_task_new_diagnose_blood_v2/', out_csv_names), index=False)

100%|███████████████████████████████████████| 100/100 [00:00<00:00, 5512.66it/s]


In [10]:
# df_pred.to_csv(os.path.join('results/gpt4_new/cross_task_new_diagnose_blood/', out_csv_names), index=False)

In [12]:
c

"**Task:**\nYou are an experienced doctor. Based on the provided patient age, demographic information and medical events, use your medical knowledge and reasoning to:\nPredict whether the patient will have first diagnosis of: 'Hypertension', within the next year after discharge from hospital.\nPredict whether the patient will have first diagnosis of: 'Hyperlipidemia', within the next year after discharge from hospital.\nPredict whether the patient will have first diagnosis of disease: 'Acute Myocardial Infarction', within the next year after discharge from hospital.\nYou are give the medical events up to the patient's discharges. Please respond 'Yes' if it is plausible and 'No' otherwise. Be aware that your confidence in diagnosis one disease might substantively influence the confidence in other diseases.\nNotice that:\nHyperlipidemia and Hypertension are well-known risk factors for cardiovascular diseases, including Acute Myocardial Infarction.\n\n{events}"

In [12]:
print(e)

**Patient age and demographic information:**
The patient was 36 years old at the prediction time.
The patient has the following demographic information: Asian,FEMALE,Not Hispanic or Latino.

**Medical Events:**
At June 12, 2014:
1 events:'oxycodone hydrochloride 5 MG Oral Tablet' recorded 
1 events:'Acetaminophen 10 MG/ML Injectable Solution' recorded 
1 events:'1 ML ketorolac tromethamine 15 MG/ML Injection' recorded 
46 events:'Heart rate' recorded with values: 83.0, 84.0, 89.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 103.0, 104.0, 105.0, 106.0, 108.0, 109.0, 117.0. 
29 events:'Systolic blood pressure' recorded with values: 129.0, 131.0, 132.0, 133.0, 134.0, 136.0, 137.0, 138.0, 141.0, 147.0, 148.0, 115.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 124.0, 125.0, 127.0. 
26 events:'Diastolic blood pressure' recorded with values: 64.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 73.0, 75.0, 76.0, 79.0, 81.0, 50.0, 51.0, 83.0, 55.0, 56.0, 59.0. 
1 events:'Inhaled oxyg

In [13]:
print(c)

**Task:**
You are an experienced doctor. Based on the provided patient age, demographic information and medical events, use your medical knowledge and reasoning to:
Predict whether the patient will have first diagnosis of: 'Hypertension', within the next year after discharge from hospital.
Predict whether the patient will have first diagnosis of: 'Hyperlipidemia', within the next year after discharge from hospital.
Predict whether the patient will have first diagnosis of disease: 'Acute Myocardial Infarction', within the next year after discharge from hospital.
You are give the medical events up to the patient's discharges. Please respond 'Yes' if it is plausible and 'No' otherwise. Be aware that your confidence in diagnosis one disease might substantively influence the confidence in other diseases.
Notice that:
Hyperlipidemia and Hypertension are well-known risk factors for cardiovascular diseases, including Acute Myocardial Infarction.

{events}


In [ ]:
cb

Tokens Used: 2334850
	Prompt Tokens: 2310850
	Completion Tokens: 24000
Successful Requests: 500
Total Cost (USD): $141.53100000000018